In [6]:
import pickle
import os
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import torch.nn.functional as F
import numpy as np
import random
import os

# initialize model 관련 error 메세지 없애는 코드(코드실행엔 문제없음)
from transformers import logging
logging.set_verbosity_error()

In [4]:
def makedirs(path):
    if not os.path.exists(path):
        os.makedirs(path)

##### the final model(initialized model)

In [ ]:
path = "..\\reviewdataset/review_posneg_theme_byjob\\"
file_list = os.listdir(path)
jobs = []
for filename in file_list:
    jobname = filename.split('_')[0]
    if len(jobs)==0:
        jobs.append(jobname)
    if jobname != jobs[-1]:
        jobs.append(jobname)

keywords = ['balance', 'salary', 'benefit', 'environment']
model_name = "kykim/bert-kor-base"

def review_sentiment(keyword, job):
    # check
    print(keyword, job)

    data_by_keyword = {}
    combined = {}

    with open(f'../reviewdataset/review_posneg_theme_byjob/{job}_pos_{keyword}.pkl', 'rb') as t:
        dev_pos_data = pickle.load(t)
        data_by_keyword[f'{job}_pos_{keyword}_data'] = dev_pos_data

    with open(f'../reviewdataset/review_posneg_theme_byjob/{job}_neg_{keyword}.pkl', 'rb') as f:
        dev_neg_data = pickle.load(f)
        data_by_keyword[f'{job}_neg_{keyword}_data'] = dev_neg_data

    combined[f'_{job}_{keyword}_data'] = data_by_keyword[f'{job}_pos_{keyword}_data'] + data_by_keyword[f'{job}_neg_{keyword}_data']
    random.shuffle(combined[f'_{job}_{keyword}_data'])

    second_values = []
    for data_point in combined[f'_{job}_{keyword}_data']:
        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
        tokenizer = BertTokenizer.from_pretrained(model_name)

        model.eval()

        tokens = tokenizer(data_point, padding=True, truncation=True, return_tensors="pt")
        
        with torch.no_grad():
            prediction = model(**tokens)

        prediction = F.softmax(prediction.logits, dim=1)

        # 긍정 스코어 점수 second_values에 extend
        second_values.extend([item[1].item() for item in prediction])
    print(second_values)

    avg = np.mean(second_values)
    print(avg)

    path = './result'
    makedirs(path)

    with open(f'{path}/{job}_{keyword}.txt', 'w') as file:
        for value in second_values:
            file.write(f'{value}\n')
    with open(f'{path}/{job}_{keyword}_avg.txt', 'w') as file:
            file.write(f'{avg}')

# run
for job in jobs:
    for keyword in keywords:
        review_sentiment(keyword, job)

##### original

In [ ]:
keyword = ['balance', 'salary', 'benefit', 'environment']

for kw in keyword:
    print(kw)
    data_by_keyword = {}
    combined = {}

    with open(f'../reviewdataset/review_posneg_theme_byjob/개발_pos_{kw}.pkl', 'rb') as t:
        dev_pos_data = pickle.load(t)
        data_by_keyword[f'개발_pos_{kw}_data'] = dev_pos_data

    with open(f'../reviewdataset/review_posneg_theme_byjob/개발_neg_{kw}.pkl', 'rb') as f:
        dev_neg_data = pickle.load(f)
        data_by_keyword[f'개발_neg_{kw}_data'] = dev_neg_data

    combined[f'_개발_{kw}_data'] = data_by_keyword[f'개발_pos_{kw}_data'] + data_by_keyword[f'개발_neg_{kw}_data']

    second_values = []
    for data_point in combined[f'_개발_{kw}_data']:
        tokens = tokenizer(data_point, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            prediction = model(**tokens)

        prediction = F.softmax(prediction.logits, dim=1)

        # 긍정 스코어 점수 second_values에 extend
        second_values.extend([item[1].item() for item in prediction])
    print(second_values)

    avg = np.mean(second_values)
    print(avg)

    makedirs('./test')

    with open(f'./test/dev_{kw}_softmax.txt', 'w') as file:
        for value in second_values:
            file.write(f'{value}\n')
    with open(f'./test/dev_{kw}_avg.txt', 'w') as file:
            file.write(f'{avg}')



        # average_second_value = sum(second_values) / len(second_values)
    
    # print(len(second_values))
    # print(average_second_value)

    # with open(f'./review_sentiment/sentiment_values/개발_{kw}_second_values.txt', 'w') as file:
    #     for value in second_values:
    #         file.write(f'{value}\n')

    # with open(f'./review_sentiment/sentiment_values/개발_{kw}_average_second_values.txt', 'w') as file:
    #     file.write(f'Average Second Value for 개발 {kw}: {average_second_value}')

##### 3rd try

In [ ]:
keyword = ['balance', 'salary', 'benefit', 'environment']

for kw in keyword:

    model_name = "kykim/bert-kor-base"
    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
    tokenizer = BertTokenizer.from_pretrained(model_name)

    model.eval()

    print(kw)
    data_by_keyword = {}
    combined = {}

    with open(f'../reviewdataset/review_posneg_theme_byjob/개발_pos_{kw}.pkl', 'rb') as t:
        dev_pos_data = pickle.load(t)
        data_by_keyword[f'개발_pos_{kw}_data'] = dev_pos_data

    with open(f'../reviewdataset/review_posneg_theme_byjob/개발_neg_{kw}.pkl', 'rb') as f:
        dev_neg_data = pickle.load(f)
        data_by_keyword[f'개발_neg_{kw}_data'] = dev_neg_data

    combined[f'_개발_{kw}_data'] = data_by_keyword[f'개발_pos_{kw}_data'] + data_by_keyword[f'개발_neg_{kw}_data']
    random.shuffle(combined[f'_개발_{kw}_data'])

    second_values = []
    for data_point in combined[f'_개발_{kw}_data']:
        tokens = tokenizer(data_point, padding=True, truncation=True, return_tensors="pt")
        
        with torch.no_grad():
            prediction = model(**tokens)

        prediction = F.softmax(prediction.logits, dim=1)

        # 긍정 스코어 점수 second_values에 extend
        second_values.extend([item[1].item() for item in prediction])
    print(second_values)

    avg = np.mean(second_values)
    print(avg)

    makedirs('./test_3rd')

    with open(f'./test_3rd/dev_{kw}_softmax.txt', 'w') as file:
        for value in second_values:
            file.write(f'{value}\n')
    with open(f'./test_3rd/dev_{kw}_avg.txt', 'w') as file:
            file.write(f'{avg}')

In [ ]:
# 3rd, 5th - initialized model